In [10]:
## 1. DataSet import
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from prepare_datasets import *
from Helper_functions import *

import tensorflow as tf
tf.compat.v1.disable_eager_execution()

X, y, feature_names, categorical_features, continuous_features, actionable_features = get_and_preprocess_cc()

X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

X_pos = X[y == 1]
X_neg = X[y == 0]

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=42)

In [11]:
from Model import NeuralNetwork

model0= NeuralNetwork(X.shape[1], 200, 2)
model1= NeuralNetwork(X.shape[1], 200, 2)
model2= NeuralNetwork(X.shape[1], 200, 2)
model3= NeuralNetwork(X.shape[1], 200, 2)
model4= NeuralNetwork(X.shape[1], 200, 2)
model5= NeuralNetwork(X.shape[1], 200, 2)
model6= NeuralNetwork(X.shape[1], 200, 2)

In [12]:
models = [model0,model1, model2, model3, model4, model5, model6]
lambdas = [0,0.05,0.1,0.15,0.2,0.25,0.3]

model_path = f"models/communities_model_0.pth"
model0.load_state_dict(torch.load(model_path))
model0.eval()  # Set to evaluation mode

# Load saved weights
for lambda_model, lamda in zip(models[1:], lambdas[1:]):
    model_path = f"models/communities_model_lambda_{lamda:.2f}.pth"
    lambda_model.load_state_dict(torch.load(model_path))
    lambda_model.eval()  # Set to evaluation mode


In [13]:
import torch.nn.functional as F
class WrappedModelForAlibi:
    def __init__(self, model):
        self.model = model
        self.model.eval()  # Important for consistent behavior

    def predict(self, x):
        with torch.no_grad():
            x_tensor = torch.tensor(x, dtype=torch.float32)
            logits = self.model(x_tensor)

            if logits.ndim == 1:
                logits = logits.unsqueeze(0)  # Ensure shape is (1, num_classes)

            probs = F.softmax(logits, dim=1)
            return probs.numpy()


In [16]:
from alibi.explainers import CEM
X_np = X.numpy()

# Wrap your model
predict_fn = WrappedModelForAlibi(model0).predict

# Feature ranges from training data
feature_min = X_np.min(axis=0)
feature_max = X_np.max(axis=0)
feature_range = (feature_min, feature_max)

cem = CEM(
    predict_fn,
    mode='PN',
    shape=(1, X_np.shape[1]),
    max_iterations=1000,
    feature_range = feature_range
)
# Fit on a sample of training data (preferably more than 1)
cem.fit(X_np)

# Explain a sample (batch size 1)
explanation = cem.explain(X_np[1:2])

print(explanation)


No PN found!


Explanation(meta={
  'name': 'CEM',
  'type': ['blackbox', 'tensorflow', 'keras'],
  'explanations': ['local'],
  'params': {
              'mode': 'PN',
              'shape': (1, 100),
              'kappa': 0.0,
              'beta': 0.1,
              'feature_range': (array([1.0005e+04, 1.6000e+00, 0.0000e+00, 2.6800e+00, 3.0000e-02,
       1.2000e-01, 4.5800e+00, 9.3800e+00, 4.6400e+00, 1.6600e+00,
       0.0000e+00, 0.0000e+00, 1.1576e+04, 3.1680e+01, 0.0000e+00,
       7.9100e+00, 4.8100e+00, 5.0000e-01, 3.4600e+00, 1.3785e+04,
       5.2370e+03, 5.4720e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 7.8000e+01, 6.4000e-01, 2.0000e-01, 2.0900e+00,
       1.6300e+00, 1.3200e+00, 2.4820e+01, 2.0500e+00, 8.6900e+00,
       1.3700e+00, 6.4800e+00, 2.1300e+00, 1.2060e+01, 3.3500e+00,
       2.8300e+00, 2.2900e+00, 3.2240e+01, 2.6110e+01, 2.7430e+01,
       3.0640e+01, 2.4420e+01, 4.1950e+01, 0.0000e+00, 0.0000e+00,
       2.0000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0

For communities and crime it is impossible to generate counterfactual for all instances because it takes alot of time and not always work for instance the above try shows that even after 1000 iterations it want able to find a PN and it took around 52 seconds. So we dropped the evaluation with communities and crimes using CEM